**Task 08: Completing missing data**

In [2]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2024-2025/master/Assignment4/course_materials"


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\richi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from rdflib import Graph, Namespace, Literal, URIRef
g1 = Graph()
g2 = Graph()
g1.parse(github_storage+"/rdf/data01.rdf", format="xml")
g2.parse(github_storage+"/rdf/data02.rdf", format="xml")

<Graph identifier=Nff1cae70468b4091b0b0fadeb894ed5b (<class 'rdflib.graph.Graph'>)>

Tarea: lista todos los elementos de la clase Person en el primer grafo (data01.rdf) y completa los campos (given name, family name y email) que puedan faltar con los datos del segundo grafo (data02.rdf). Puedes usar consultas SPARQL o iterar el grafo, o ambas cosas.

In [17]:
from rdflib.plugins.sparql import prepareQuery
from rdflib.namespace import RDF, RDFS

ns = Namespace("http://data.org#")
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")

q1 = prepareQuery('''
    SELECT ?person ?givenName ?familyName ?email WHERE {
        ?person rdf:type ns:Person .
        OPTIONAL { ?person VCARD:Given ?givenName . }
        OPTIONAL { ?person VCARD:Family ?familyName . }
        OPTIONAL { ?person VCARD:EMAIL ?email . }
    }
''', initNs={"rdf": RDF, "ns": ns, "VCARD": VCARD}
)

for r in g1.query(q1):
  print(r.person)

http://data.org#SaraJones
http://data.org#JohnSmith
http://data.org#JohnDoe
http://data.org#HarryPotter


In [20]:
person_data = {}
for r in g1.query(q1):
    person_data[r.person] = {
        "givenName": r.givenName,
        "familyName": r.familyName,
        "email": r.email
    }

for person_uri in person_data.keys():
    # Obtener el nombre dado, apellido y correo electrónico desde el segundo grafo
    query2 = prepareQuery('''
        SELECT ?givenName ?familyName ?email WHERE {
            ?person rdf:type ns:Person .
            FILTER (?person = ?personUri)
            OPTIONAL { ?person VCARD:Given ?givenName . }
            OPTIONAL { ?person VCARD:Family ?familyName . }
            OPTIONAL { ?person VCARD:EMAIL ?email . }
        }
    ''', initNs={"rdf": RDF, "ns": ns, "VCARD": VCARD}
    )

    # Ejecutar la consulta
    for r in g2.query(query2, initBindings={"personUri": person_uri}):
        # Completar los campos que faltan
        if not person_data[person_uri]["givenName"]:
            person_data[person_uri]["givenName"] = r.givenName
        if not person_data[person_uri]["familyName"]:
            person_data[person_uri]["familyName"] = r.familyName
        if not person_data[person_uri]["email"]:
            person_data[person_uri]["email"] = r.email

for person, details in person_data.items():
    print(f"Person: {person}")
    print(f"Given Name: {details['givenName']}")
    print(f"Family Name: {details['familyName']}")
    print(f"Email: {details['email']}")
    print("---------------------------")


Person: http://data.org#SaraJones
Given Name: Sara
Family Name: Jones
Email: sara.jones@data.org
---------------------------
Person: http://data.org#JohnSmith
Given Name: John
Family Name: Smith
Email: j.smith@data.org
---------------------------
Person: http://data.org#JohnDoe
Given Name: John
Family Name: Doe
Email: doe@data.org
---------------------------
Person: http://data.org#HarryPotter
Given Name: Harry
Family Name: Potter
Email: hpotter@hogwarts.org
---------------------------
